# Game Map Analysis

## Preparation

In [4]:
import re
from ipywidgets import FloatProgress
from IPython.display import display
import matplotlib.image as image
%run "../Utilities/Preparation.ipynb"

C:\Users\Mikael\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (18,21,22,28,32,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Mikael\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (14,15,16,17,18,20,21,22,25,26,27,30,33,34,35,36,37,38,39,40,41,42,43) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Functions

In [12]:
# eg _types = ['death', 'reach', 'add', 'craft', 'remove', 'select', 'selectmenu']

def filterAndLocateEvents(_types):
    _mapRelevantColumns = ['type', 'section', 'coordinates']
    _locatedEventsDF = rdf152.loc[:,_mapRelevantColumns].dropna()

    # types
    _locatedEventsDF = _locatedEventsDF[_locatedEventsDF['type'].isin(_types)]

    # section
    _locatedEventsDF = _locatedEventsDF[(_locatedEventsDF['section'].str.startswith('tutorial1'))]

    # x,y
    _locatedEventsDF['x'] = _locatedEventsDF['coordinates']
    _locatedEventsDF['y'] = _locatedEventsDF['coordinates']

    _f = FloatProgress(min=0, max=len(_locatedEventsDF.index))
    display(_f)

    for index in _locatedEventsDF.index:
        _coordinates = re.findall('-*\d+', _locatedEventsDF['coordinates'][index])
        _locatedEventsDF['x'][index] = int(_coordinates[0])
        _locatedEventsDF['y'][index] = int(_coordinates[1])
        _f.value += 1

    _locatedEventsDF = _locatedEventsDF.drop('coordinates', 1)

    return _locatedEventsDF

In [ ]:
def initializeConstants():
    gameStartCoordinates = [-229,-608]
    gameEndCoordinates = [441,-472]

    # only for '../../images/map.tutorial1.png'
    imgStartCoordinates = [833,432]
    imgEndCoordinates = [1333,334]


    scaleX = (gameStartCoordinates[0]-gameEndCoordinates[0])/(imgStartCoordinates[0]-imgEndCoordinates[0])
    offsetX = gameStartCoordinates[0] - (imgStartCoordinates[0]*scaleX)
    scaleY = (gameStartCoordinates[1]-gameEndCoordinates[1])/(imgStartCoordinates[1]-imgEndCoordinates[1])
    offsetY = gameStartCoordinates[1] - (imgStartCoordinates[1]*scaleY)

def getGraphPosition(imgPos):
    _x = getXGraphPosition(imgPos[0])
    _y = getYGraphPosition(imgPos[1])
    return [_x, _y]

def getXGraphPosition(xImg):
    return scaleX*xImg + offsetX

def getYGraphPosition(yImg):
    return scaleY*yImg + offsetY

In [13]:
def plotLocatedEvents(_XYEventsDF, _types):
    
    initializeConstants()

    plt.figure(figsize=(18,10))

    # background map image display
    _mapBackground = image.imread('../../images/map.tutorial1.png')

    # background image dimensions
    _minX = 0
    _maxX = _mapBackground.shape[1]
    _minY = _mapBackground.shape[0]
    _maxY = 0

    plt.imshow(_mapBackground, aspect='auto', zorder=-1, \
               extent=(getXGraphPosition(_minX), getXGraphPosition(_maxX), getYGraphPosition(_minY), getYGraphPosition(_maxY)), \
               alpha=0.5)


    _colors = ['black', 'red', 'blue', 'green', 'orange', 'purple', 'brown', 'pink', 'olive', 'cyan', ]
    _alphas = [ 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, ]
    _scatters = {}

    for i, t in enumerate(_types):
      _pts = _XYEventsDF[_XYEventsDF['type'] == t]
      x = _pts['x']
      y = _pts['y']

      _scatters[t] = plt.plot(x, y, 'o', c=_colors[i], lw=0, alpha=_alphas[i])

    plt.legend([ x[0] for x in _scatters.values()], _scatters.keys())

    # graph dimensions and extrema on tutorial1
    # _minX=-1152
    # _maxX=539
    # _minY=-1161
    # _maxY=-100
    plt.xlim([-1200, 600])
    plt.ylim([-1200, -200])
    
    plt.show()